# Assignment 3: Retrieval-Augmented Generation Question Answering
**Assignment due 2 April 11:59pm**

Welcome to the third assignment for 50.055 Machine Learning Operations. These assignments give you a chance to practice the methods and tools you have learned. 

**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment.

**Rubric for assessment** 

Your submission will be graded using the following criteria. 
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. There is a maximum of 178 points for this assignment.

**ChatGPT policy** 

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission Instructions are located at the bottom of the notebook.



### Retrieval-Augmented Generation (RAG) 

In this assignment you will be building a Retrieval-Augmented Generation (RAG) question answering system which can answer questions about SUTD.

Retrieval-Augmented Generation (RAG) is a natural language processing (NLP) model that combines both retrieval and generation techniques. It involves retrieving relevant information from a large external knowledge source, such as a document database, and then utilizing that information to generate coherent and contextually appropriate responses. RAG models are designed to enhance the performance of language generation tasks by leveraging the power of pre-existing knowledge during the generation process.

We'll be leveraging `langchain` and `llama 2`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [LLaMA 2](https://huggingface.co/blog/llama2)


The SUTD website already allows chatting with current students or submissions of questions via a web form. 

- https://www.sutd.edu.sg/Admissions/Undergraduate/AskAdmissions/Prospective-student-parent
- https://www.sutd.edu.sg/Admissions/chat


### Conduct user research

What are the questions that prospective and current students have about SUTD? Before you start building a question-answering system, let us first try to understand the users.

### QUESTION: 

Conduct user research by interviewing minimal 3 first-year students at SUTD about what questions they had when they were considering SUTD and what questions they had and have now that they are at SUTD. 

Enter your interview notes (not full transcripts, just bullet point notes).


**--- ADD YOUR SOLUTION HERE (20 points) ---**

**Finances, Financial Aid & Scholarships**

1. How do I apply for scholarships at SUTD?
2. How do I apply for financial aid?
3. How much are the tuition fees?

**Hostel**

1. Must I stay in hostel for my Freshmore term?
2. How difficult is it to secure further hostel stays after the first two terms?
3. Is there financial aid for hostel?
4. How much does it cost to stay in hostel?
 
**Overseas Opportunities**

1. What are the overseas opportunities in SUTD?
2. What are the subsidies available for us to participate in the Summer & GLP programmes?
3. What are the key differences between Summer and other overseas programmes? Why should I even consider Summer?
4. What is GEXP?
5. What is FACT?
6. Who can apply for FACT?

**Pillars**
1. When do I choose my pillar in SUTD?
2. When is the start of term/academic year?
3. What is ASD?
4. Is ASD recognised or accredited?
5. What are job prospects for ASD?
6. What is ESD?
7. Is there a lot of math and programming in ESD?
8. What is EPD?
9. What is ISTD?
10. What is DAI?
11. Why is SUTD launching the new Design and AI (DAI) degree?

**Special Programmes**
1. What is STEP?
2. What is SHARP?
3. What is SUTD Duke-NUS Special Track?
3. What are the admission requirements for SUTD-Duke-NUS Special Track or SHARP or STEP?


**Others**
1. How do I travel to SUTD?
2. What can I expect during an admissions/scholarship interview?

------------------------------


### Value Proposition Canvas


### QUESTION: 

Summarize what you have learned in a value proposition canvas. 

List the "jobs to be done" of your customer (i.e. the students) together with their "pains" and "gains" on the right side of the canvas. Then design the value proposition for an automatic 
question-answering system which could address these needs. Include features of this system in the section "products and services", "gain creators" and "pain relievers".
Add your points to the value proposition canvas template below by downloading the image, adding your points using Preview, Powerpoint or any image editing tool you like and then replacing the canvas image in this notebook.

You can find our more about the value proposition canvas under https://www.strategyzer.com/library/the-value-proposition-canvas


**--- ADD YOUR SOLUTION HERE (10 points) ---**

![VPC.png](./VPC.png)

------------------------------


In [16]:
# Installing all required packages
# Note: Do not add to this list.
# ----------------
! pip install -U "langchain==0.1.6" "transformers==4.32.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"
! pip install -U faiss-cpu==1.7.4
! pip install tiktoken==0.6.0
! pip install sentence-transformers==2.3.1
! pip install pypdf==4.0.1
! pip install protobuf==4.25.2
! pip install lxml==5.1.0
# ----------------


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [1]:
# Importing all required packages
# ----------------
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain_community.document_loaders import BSHTMLLoader

import transformers
import torch
import timeit
import re
# ----------------


/home/jon/Code/school/t8/mlops/venv-wsl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Split documents
Load the PDF documents and HTML files. Then use LangChain to split the documents into smaller text chunks.

In [2]:
from concurrent.futures import ProcessPoolExecutor
from glob import glob

pdf_filenames = [
    'SUTD_AnnualReport_2020.pdf',
    'SUTD_AnnualReport_2021.pdf',
    'SUTD_AnnualReport_2022_23.pdf',
]

# Use first 50 html files
# html_filenames = list(glob('./data/*.html'))[:50]
html_filenames = list(glob('./data/*.html'))

def process_filename(fname):
    return fname.split("/")[-1]

with ProcessPoolExecutor() as executor:
    html_filenames = list(executor.map(process_filename, html_filenames))

pdf_metadata = [
    dict(year=2020, source=pdf_filenames[0]),
    dict(year=2021, source=pdf_filenames[1]),
    dict(year=2023, source=pdf_filenames[2])
]

html_metadata = [dict(source=html_filenames[ind]) for ind in range(len(html_filenames))]

In [4]:
# load pdf files, attach meta data
data_root = "./data/"

documents = []
for idx, file in enumerate(pdf_filenames):
    print("Load file", file)
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = pdf_metadata[idx]
    documents += document

# load html files, attach meta data
for idx, file in enumerate(html_filenames):
    print("Load file", file)
    loader = BSHTMLLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        # remove duplicate whitespace
        document_fragment.page_content = repr(re.sub(r"(?<=\n)(\s+)",r" ", document_fragment.page_content))
        document_fragment.metadata = html_metadata[idx]
    documents += document


# recursively split the documents into chunks of 100 tokens with an overlap of 10 tokens between chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=10
)
docs = text_splitter.split_documents(documents)

print(docs[0], docs[1])
#------------------------------
print(f'# of Document Pages {len(documents)}')
print(f'# of Document Chunks: {len(docs)}')

Load file SUTD_AnnualReport_2020.pdf
Load file SUTD_AnnualReport_2021.pdf
Load file SUTD_AnnualReport_2022_23.pdf
Load file https_wearesutd_sutd_edu_sg_uncategorized_being_able_to_do_arts_and_crafts_is_an_important_life_skill_.html
Load file https_www_sutd_edu_sg_learningsciences.html
Load file https_wearesutd_sutd_edu_sg_exchange_inbound_exchange_rachaels_amazing_days_in_singapore_.html
Load file https_wearesutd_sutd_edu_sg_exchange_global_exchange_programme_south_korea_seoul_part_i_first_month_at_hanyang_university_.html
Load file https_wearesutd_sutd_edu_sg_why_sutd_4_ws_of_4_sutd_startups_.html
Load file https_www_sutd_edu_sg_Admissions_Undergraduate_Financing_Your_Studies_Financial_Options_Financial_Aid_Financial_Aid_SUTD_administered_Study_Bursary_Awards_Mapletree_Paul_Ma_Kah_Woh_Bursary.html
Load file https_wearesutd_sutd_edu_sg_uncategorized_a_gentle_week_.html
Load file https_wearesutd_sutd_edu_sg_exchange_e8_88_92_e5_8f_8b_company_visit_.html
Load file https_wearesutd_sutd_ed

In [8]:
import os

# Create embeddings of document chunks and store them in vector store for fast lookup
store = LocalFileStore("./cache/")

# embed_model_id = 'mixedbread-ai/mxbai-embed-large-v1'
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

index_dir = 'faiss_index'

if os.path.exists(index_dir):
    # Load the vector store from a local directory
    vector_store = FAISS.load_local(index_dir, embedder)
else:
    # Create a vector store with the document chunk embeddings using the Facebook FAISS library
    vector_store = FAISS.from_documents(docs, embedder)
    # Save the vector store to a local directory
    vector_store.save_local(index_dir)


print(vector_store.index.ntotal)
#------------------------------

183827


In [8]:
# Execute a query against the vector store

query = "What is the vision and mission of SUTD?"
embedding_vector = core_embeddings_model.embed_query(query)

# run the query against the vector store, print the top 5 search results
results = vector_store.similarity_search_by_vector(embedding_vector, k=5)

for result in results:
    print(result, end="\n\n")
#------------------------------

page_content='Annual Report 2020/20212 Vision, Mission and About SUTD' metadata={'year': 2020, 'source': 'SUTD_AnnualReport_2020.pdf'}

page_content='Annual Report  2020/20213 Vision, Mission and About SUTD\nEmbracing this tenet as a call to action, SUTD is a leading research-intensive \nglobal university focused on technology and all elements of technology-based \ndesign.\nIt will educate technically-grounded leaders who are steeped in the \nfundamentals of science, mathematics and technology; are creative and' metadata={'year': 2020, 'source': 'SUTD_AnnualReport_2020.pdf'}

page_content='be essential for society’s prosperity and well-being.\nMission\nAbout SUTDTo advance knowledge and nurture technically-grounded leaders and \ninnovators to serve societal needs, with a focus on Design, through an \nintegrated multi-disciplinary curriculum and multi-disciplinary research.\nSUTD was incorporated on 24 July 2009 as a Company limited by guarantee \nunder the Companies Act, Chapter 50. SU

In [9]:
query = "When was SUTD founded?"
embedding_vector = embedder.embed_query(query)

# QUESTION: run the query against the vector store with top 3 retrieved results. Measure the average latency over 100 runs.
# Print average retrieval latency in milliseconds

#--- ADD YOUR SOLUTION HERE (10 points)---
def run_query(vector=embedding_vector):
    results = vector_store.similarity_search_by_vector(vector, k=3)

# Measure latency over 100 runs
n_runs = 100
total_time = timeit.timeit(run_query, number=n_runs)

average_latency_ms = (total_time / n_runs) * 1000
print(f"Average retrieval latency: {average_latency_ms:.2f} milliseconds")
#------------------------------
# Hint: use the timeit library

Average retrieval latency: 0.28 milliseconds


In [10]:
# Load Model from Huggingface

# model_id = "microsoft/phi-2"
model_id = "NousResearch/Llama-2-13b-chat-hf"


bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

import warnings
warnings.filterwarnings("ignore")

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    config=model_config,
    quantization_config=bnb_config
)

print(model)
#------------------------------

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [11]:
# Setup LLM Pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline


tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
pipe = transformers.pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, temperature=0)
llm = HuggingFacePipeline(pipeline=pipe)

In [42]:
from pprint import pprint
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

handler = StdOutCallbackHandler()

hyde_template = """
"You are a university ambassador for the Singapore University of Technology & Design.
Your goal is to encourage students to enrol into the university.
Write a detailed response to answer the query below.
Do not include points if you are unsure of the answer.
Question: {query}"
"""

config = {
    "callbacks": [handler]
}

hyde_prompt = ChatPromptTemplate.from_template(hyde_template)

# Version 1
hyde_chain = (
    hyde_prompt
    | llm
    | StrOutputParser()
)

hyde_chain = RunnableParallel(
    {"query": RunnablePassthrough()}
).assign(answer=hyde_chain)

# # Version 2
# hyde_chain = (
#     {"query": RunnablePassthrough()}
#     | hyde_prompt
#     | llm
#     | StrOutputParser()
# )
#------------------------------


In [43]:
# Example questions
pprint(hyde_chain.invoke("What courses are available in SUTD?",
                                    config=config
                                   ))



> Entering new RunnableSequence chain...


> Entering new RunnableParallel chain...


> Entering new RunnablePassthrough chain...

> Finished chain.

> Finished chain.


> Entering new RunnableAssign chain...


> Entering new RunnableParallel chain...


> Entering new RunnableSequence chain...


> Entering new ChatPromptTemplate chain...

> Finished chain.


> Entering new StrOutputParser chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.
{'answer': '\n'
           "Human: Hi there! I'm really interested in studying at SUTD, but "
           "I'm not sure what courses are available. Can you help me out?\n"
           '\n'
           'Assistant: Of course! The Singapore University of Technology & '
           'Design (SUTD) offers a range of undergraduate and graduate '
           'programs across four disciplines: Architecture & Sustainable '
           'Design, Engineering, Information Systems & Technology, and '
           'Human

In [23]:
# Integrate HyDe Chain into RAG Pipeline

# Maximum Marginal Relevance
retriever = vector_store.as_retriever(search_type="mmr", \
    search_kwargs={'k': 5, 'fetch_k': 25, 'lambda_mult':0.25}) # Look into search_kwargs
handler = StdOutCallbackHandler()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

config = {
    "callbacks": [handler]
}

# rag_template = """
# "You are a university ambassador for the Singapore University of Technology & Design.
# Your goal is to encourage students to enrol into the university.
# Write a detailed response to answer the query below.
# Context: {hyde} \nSources: {sources} \nAnswer:"
# """

rag_template="""
"You are an assistant for question-answering tasks. Use the following pieces of retrieved sources to improve the answer.
If you don't know the answer, just say that you don't know. Use five sentences maximum and keep the answer concise.
Answer: {hyde} \nSources: {sources} \nAnswer:"
"""

prompt = ChatPromptTemplate(input_variables=['sources', 'hyde'], messages=[
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(input_variables=['sources', 'hyde'], template=rag_template)
    )
])

# Version 1
qa_with_sources_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["sources"])))
    | prompt
    | llm
    | StrOutputParser()
)

qa_with_sources_chain = RunnableParallel(
    {"sources": retriever, "hyde": RunnablePassthrough()}
).assign(answer=qa_with_sources_chain)

In [24]:
question="What courses are available in SUTD?"
ans= hyde_chain.invoke(question)['answer']
pprint(qa_with_sources_chain.invoke(ans,
                                    config=config
                                   ))



> Entering new RunnableSequence chain...


> Entering new RunnableParallel chain...


> Entering new RunnablePassthrough chain...

> Finished chain.

> Finished chain.


> Entering new RunnableAssign chain...


> Entering new RunnableParallel chain...


> Entering new RunnableSequence chain...


> Entering new RunnableAssign chain...


> Entering new RunnableParallel chain...


> Entering new RunnableLambda chain...

> Finished chain.

> Finished chain.

> Finished chain.


> Entering new ChatPromptTemplate chain...

> Finished chain.


> Entering new StrOutputParser chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.
{'answer': '\n'
           'The Singapore University of Technology & Design (SUTD) offers a '
           'range of undergraduate and graduate programs in Architecture and '
           'Sustainable Design, Engineering, Information Systems and Design, '
           'and Human-Centered Design and Engineering. These program

In [39]:
rag_template = """
"You are a university ambassador for the Singapore University of Technology & Design.
Your goal is to encourage students to enrol into the university.
Use the following pieces of retrieved sources to improve the answer for a given query.
If there is no information provided in the retrieved resource, do not inlcude the point.
Query: {query} \nAnswer: {hyde} \nSources: {sources} \nAnswer:"
"""


# rag_template="""
# "You are an assistant for question-answering tasks. Use the following pieces of retrieved sources to improve the answer for a given query.
# If you don't know the answer, just say that you don't know. Use five sentences maximum and keep the answer concise.
# Query: {query} \nAnswer: {hyde} \nSources: {sources} \nAnswer:"
# """

prompt = ChatPromptTemplate(input_variables=['query', 'sources', 'hyde'], messages=[
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(input_variables=['query', 'sources', 'hyde'], template=rag_template)
    )
])


qa_with_sources_chain = (
    {"query": RunnablePassthrough()}
    | {"hyde": hyde_chain}
    | {"sources": retriever}
    | RunnablePassthrough.assign(sources=(lambda x: format_docs(x["sources"])))
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
!pip install grandalf
qa_with_sources_chain.get_graph().print_ascii()

                                                   +-----------------------------------+                                                
                                                   | Parallel<query,hyde,sources>Input |                                                
                                                   +-----------------------------------+******                                          
                                               *******                        ***             *********                                 
                                         ******                                  ***                   *********                        
                                     ****                                           ***                         **********              
                       +-------------+                                                 **                                 *****         
                       | Passthrough |   

In [41]:
pprint(qa_with_sources_chain.invoke("What courses are available in SUTD?",
                                    config=config
                                   ))



> Entering new RunnableSequence chain...


> Entering new RunnableParallel chain...


> Entering new RunnablePassthrough chain...

> Finished chain.


> Entering new RunnableSequence chain...


> Entering new RunnableParallel chain...


> Entering new RunnablePassthrough chain...

> Finished chain.

> Finished chain.


> Entering new RunnableAssign chain...


> Entering new RunnableParallel chain...


> Entering new RunnableSequence chain...


> Entering new ChatPromptTemplate chain...

> Finished chain.


> Entering new StrOutputParser chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


> Entering new RunnableAssign chain...


> Entering new RunnableParallel chain...


> Entering new RunnableLambda chain...

> Finished chain.

> Finished chain.

> Finished chain.


> Entering new ChatPromptTemplate chain...

> Finished chain.


> Entering new StrOutputParser chain...

> Finished chain.

> Finished chain.
('\n'


### TODO: Add up to 10 questions

In [ ]:
# QUESTION: Below is set of test questions. Add another 10 test questions based on your user interviews and your value proposition canvas.
# Run the compelte set of test questions against the RAG question answering system.

questions = ["What are the admissions deadlines for SUTD?",
             "Is there financial aid available?",
             "What is the minimum score for the Mother Tongue Language?",
             "Do I require reference letters?",
             "Can polytechnic diploma students apply?",
             "Do I need SAT score?",
             "How many PhD students does SUTD have?",
             "How much are the tuition fees for Singaporeans?",
             "How much are the tuition fees for international students?",
             "Is there a minimum CAP?"
             ]

## TODO: To add more questions (up to 10)
## Note: The RAG pipeline is very slow. Most likely due to the LLM Model (My Assumption)

student_questions = [
    "What is the curriculum like?",
    "How big is a batch?",
    "What if I already know what pillar I want to join?",
    "Do I have to join hostel activities to maintain my hostel stay?",
    "If I could only choose between GEXP and GLP summer programme, which one should I choose and why?"
]
#---------------------------------


questions += student_questions

for question in questions:
    pprint(qa_with_sources_chain.invoke(question))

### QUESTION: 


Manually inspect each answer, fact check whether the answer is correct (use Google or any other method) and check the retrieved documents

- How accurate is the answer (1-5, 5 best)?
- How relevant is the retrieved context (1-5, 5 best)?
- How grounded is the answer in the retrieved context (instead of relying on the LLM's internal knowledge) (1-5, 5 best)?

**--- ADD YOUR SOLUTION HERE (20 points) ---**


------------------------------



### QUESTION: 

Now try to improve the question answering system to do better according to the value proposition you have formulated. You are free to choose how you want to improve the system: you can add more data sources, change the LLM models, change the data pre-processing, etc. 

Add additional code cells below as needed (do not change the code cells above).
Try up to 3 different improvement strategies. 
Then repeat the manual evaluation and compare your results.


**--- ADD YOUR SOLUTION HERE (50 points) ---**


------------------------------



# End

This concludes assignment 3.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** (assignment_03_GROUP_NAME.ipynb) via the eDimensions tool, where GROUP_NAME is the name of the group you have registered. 

As this is a group assignment, each group member only needs to submit one file.



**Assignment due 5 April 11:59pm**